In [ ]:
import gymnasium as gym
%env MUJOCO_GL=egl
from datasets import load_from_disk, concatenate_datasets


In [ ]:
dataset_1 = load_from_disk("BC-MLP-MSE/datasets/grasp_100_2024-09-06_17-03-47.hf")
dataset_2 = load_from_disk("BC-MLP-MSE/datasets/50_single_2024-09-16_15-07-50.hf")

d1_eps = range(10)
d2_eps = range(40)

dataset_1_filtered = dataset_1.filter(lambda example: example["episode_index"] in d1_eps)
dataset_2_filtered = dataset_2.filter(lambda example: example["episode_index"] in d2_eps)



In [ ]:
dataset_1_filtered = dataset_1_filtered.map(lambda example, idx: {"episode_index": d1_eps.index(example["episode_index"]), "index": idx}, with_indices=True)
dataset_2_filtered = dataset_2_filtered.map(lambda example, idx: {"episode_index": d2_eps.index(example["episode_index"]) + dataset_1_filtered["episode_index"][-1] + 1, "index": idx + dataset_1_filtered["index"][-1] + 1}, with_indices=True)

In [ ]:
dataset_2_filtered["index"][-1]

In [ ]:
dataset = concatenate_datasets([dataset_1_filtered, dataset_2_filtered])


In [ ]:
assert(dataset["index"] == list(range(len(dataset))))
assert(list(set(dataset["episode_index"])) == list(range(len(d1_eps) + len(d2_eps))))

In [ ]:
import datetime
dataset.save_to_disk(f"BC-MLP-MSE/datasets/10_diverse_40_single_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.hf")
